In [0]:
!pip install lark-parser --user



     |████████████████████████████████| 276kB 4.8MB/s 
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62514 sha256=1eb167175b473d6009ef90d1fc2d073b71180a5043cb009e793df9ca28488adc
  Stored in directory: /root/.cache/pip/wheels/01/a2/30/ebae6ffa73cf3aa1c972a24d4c78388afd910f91e43bf554aa
Successfully built lark-parser


In [0]:
from pprint import pprint
from collections import ChainMap
from lark import Lark, InlineTransformer, Tree, Token
Tree._repr_html_ = lambda t: '<pre>%s</pre>' % t.pretty()



ModuleNotFoundError: ignored

In [0]:
c = Lark(r"""
?start  : value

?value  : exp+

?exp    : if";"
        | else";"
        | assign";"
        | boolop";"

if     : "if""("boolop")""{"/exp+/"}"

else   : "else""("boolop")""{"/exp+/"}" 

assign  : NAME"="bool
        | NAME"="boolop
        | NAME"="NUMBER

bool    : "true"   -> true
        | "false"  -> false

boolop  : boolval OPER boolval
        


boolval : NUMBER
        | bool
        | NAME


// Terminais
NUMBER  : /\d+(\.\d+)?/ 
NAME    : /[A-Za-z]+/
OPER    : /[><=][=]?/
%ignore /\s+/


""")


In [0]:
replace = {}
class BOOLTransformer(InlineTransformer):
    numbers = float
    
    def assign(self, var , val):
        replace[var] = val

    def boolop(self, val1, op, val2):
        return (val1, op, val2)

    def true(self):
        return True
    
    def false(self):
        return False
    


In [0]:
def eval(expr):
    if isinstance(expr, (float, int)):
        return expr
    
    elif isinstance(expr, str):
        return replace[expr]

    head, *args = expr
    if head == '>':
        x, y = args
        return eval(x) > eval(y)
    elif head == '<':
        x, y = args
        return eval(x) < eval(y)
    elif head == '<=':
        x, y = args
        return eval(x) <= eval(y)
    elif head == '>=':
        x, y = args
        return eval(x) >= eval(y)
    elif head == '==':
        x, y = args
        return eval(x) == eval(y)
    else:
        raise ValueError('argumento inválido : %r' % head)

In [0]:
boool = c.parse('2>4;')
boo = BOOLTransformer().transform(tree)
value = eval(boo)
